In [ ]:
import sys
sys.path.insert(0, '..')
from src.models.tgt_parser.struct.d1_pcfg import D1PCFG
from src.models.tgt_parser.struct.pcfg import PCFG
from pytorch_memlab import LineProfiler
import torch
from collections import Counter

In [ ]:

# B = 3
# N = 20
# TGT_PT = 8
# SRC_PT = 7
# TGT_NT = 6
# SRC_NT = 6
# r = 12

# NT = TGT_NT * SRC_NT
# T = TGT_PT * SRC_PT
# # lens = [max(2, N - i) for i in range(B)]
# lens = [N -1 for i in range(B)]

# slr = torch.rand(B, r, SRC_NT, SRC_NT + SRC_PT, SRC_NT + SRC_PT)
# slr[..., :SRC_NT, :SRC_NT] /= slr[..., :SRC_NT, :SRC_NT].sum((3, 4), keepdim=True)
# slr[..., SRC_NT:, :SRC_NT] /= slr[..., SRC_NT:, :SRC_NT].sum((3, 4), keepdim=True)
# slr[..., :SRC_NT, SRC_NT:] /= slr[..., :SRC_NT, SRC_NT:].sum((3, 4), keepdim=True)
# slr[..., SRC_NT:, SRC_NT:] /= slr[..., SRC_NT:, SRC_NT:].sum((3, 4), keepdim=True)

# params = {
#     "term": torch.randn(B, N, T).log_softmax(-1).requires_grad_(True),
#     "root": torch.randn(B, NT).log_softmax(-1).requires_grad_(True),
#     "head": torch.randn(B, NT, r).softmax(-1).requires_grad_(True),
#     "left": torch.randn(B, r, TGT_NT + TGT_PT).softmax(-1).requires_grad_(True),
#     "right": torch.randn(B, r, TGT_NT + TGT_PT).softmax(-1).requires_grad_(True),
#     "slr": slr.requires_grad_(),
# }

# pcfg = D1PCFGFlex(TGT_NT, TGT_PT)
# nll = pcfg(params, lens)

# pcfg_ref = PCFG()
# nll_ref = pcfg_ref(D1PCFGFlex.get_pcfg_rules(params, TGT_NT), lens)
# print(nll, nll_ref)
# assert torch.allclose(nll, nll_ref)

# m1 = pcfg(params, lens, marginal=True)
# assert torch.allclose(
#     m1.sum((1, 2, 3, 4)),
#     torch.tensor([item * 2 - 1 for item in lens], dtype=torch.float),
# )

# m2 = pcfg_ref(D1PCFGFlex.get_pcfg_rules(params, TGT_NT), lens, marginal=True)[-1]
# assert torch.allclose(
#     m1.diagonal(2, dim1=1, dim2=2).sum((1, 2)), m2[:, 0, :-1].sum(-1)
# )


In [ ]:
B = 1
N = 3
VOCAB = 3
TGT_PT = 4
SRC_PT = 3
TGT_NT = 4
SRC_NT = 3
r = 2

NT = TGT_NT * SRC_NT
T = TGT_PT * SRC_PT
lens = [max(2, N - i) for i in range(B)]

slr = (
    torch.randn(B, r, SRC_NT, SRC_NT, SRC_NT)
    .view(B, r * SRC_NT, -1)
    .softmax(-1)
    .view(B, r, SRC_NT, SRC_NT, SRC_NT)
)
params = {
    "term": torch.randn(B, T, VOCAB).log_softmax(-1).requires_grad_(True),
    "root": torch.randn(B, NT).log_softmax(-1).requires_grad_(True),
    "head": torch.randn(B, NT, r).softmax(-1).requires_grad_(True),
    "left": torch.randn(B, r, TGT_NT + TGT_PT).softmax(-1).requires_grad_(True),
    "right": torch.randn(B, r, TGT_NT + TGT_PT).softmax(-1).requires_grad_(True),
    "slr": slr,
}

num_samples = 100000
max_len = 4

pcfg = D1PCFG(TGT_NT, TGT_PT)
pred = pcfg.sampled_decoding(
    params,
    [[None] * SRC_NT for _ in range(B)],
    TGT_NT,
    [[None] * SRC_PT for _ in range(B)],
    TGT_PT,
    False,
    num_samples,
    max_length=max_len,
)
# print(pcfg(params, lens))
new_pred = []

for inst in pred:
    new_pred_inst = []
    for item, type, score in inst:
        new_pred_inst.append(item)
    new_pred.append(new_pred_inst)
pred = new_pred

pcfg_ref = PCFG()
params2 = D1PCFG.get_pcfg_rules(params, TGT_NT)
pred_ref = pcfg_ref.sampled_decoding(
    params2,
    [[None] * SRC_NT for _ in range(B)],
    TGT_NT,
    [[None] * SRC_PT for _ in range(B)],
    TGT_PT,
    False,
    num_samples,
    max_length=max_len,
)
# print(pred_ref(params2, lens))
new_pred = []

for inst in pred_ref:
    new_pred_inst = []
    for item, *_ in inst:
        new_pred_inst.append(item)
    new_pred.append(new_pred_inst)
pred_ref = new_pred

# assert torch.allclose(nll, nll_ref)

# m1 = pcfg(params, lens, marginal=True)
# assert torch.allclose(
#     m1.sum((1, 2, 3, 4)),
#     torch.tensor([item * 2 - 1 for item in lens], dtype=torch.float),
# )

# m2 = pcfg_ref(D1PCFGFlex.get_pcfg_rules(params, TGT_NT), lens, marginal=True)[-1]
# assert torch.allclose(
#     m1.diagonal(2, dim1=1, dim2=2).sum((1, 2)), m2[:, 0, :-1].sum(-1)
# )


In [ ]:
count = Counter(''.join(map(str, item)) for item in pred[0])
count_ref = Counter(''.join(map(str, item)) for item in pred_ref[0])

In [ ]:
count.most_common(5)

In [ ]:
count_ref.most_common(5)

In [ ]:
x = torch.tensor([[1, 1, 1, 0]])
n = x.size(1)
terms = params['term'].unsqueeze(1).expand(B, n, T, params['term'].size(2))
x_expand = x.unsqueeze(2).expand(B, n, T).unsqueeze(3)
terms = torch.gather(terms, 3, x_expand).squeeze(3)
params3 = {**params}
params3['term'] = terms
nll = pcfg(params3, [x.shape[1]])

In [ ]:
(-nll).exp() * 100000

In [ ]:
count['1110'], count_ref['1110']